# Claire Voyant - Problem 898

Claire Voyant is a teacher playing a game with a class of students.  
A fair coin is tossed on the table. All the students can see the outcome of the toss, but Claire cannot.  
Each student then tells Claire whether the outcome is head or tail. The students may lie, but Claire knows the probability that each individual student lies. Moreover, the students lie independently.  
After that, Claire attempts to guess the outcome using optimal strategy.  

For example, for a class of four students with lying probabilities $20\%,40\%,60\%,80\%$, Claire guesses correctly with probability 0.832.  

Find the probability that Claire guesses correctly for a class of 51 students each lying with a probability of $25\%, 26\%, \dots, 75\%$ respectively.  

Give your answer rounded to 10 digits after the decimal point.
## Solution.

Claire guesses **H** iff  (excluidng ties, then she guesses it with probability 1/2)

$$
\frac{\mathbb{P}(H \mid \text{seq})}{\mathbb{P}(T \mid \text{seq})} > 1,
$$

which, by Bayes' theorem, is equivalent to  

$$
\frac{\mathbb{P}(\text{seq} \mid H)}{\mathbb{P}(\text{seq} \mid T)} > 1.
$$

Now, the probability of a correct guess is  

$$
\begin{align*}
\mathbb{P}(\text{correct}) 
&= \mathbb{P}(\text{correct} \mid H) \\[6pt]
&= \mathbb{P}\!\left( \frac{\mathbb{P}(\text{seq} \mid H)}{\mathbb{P}(\text{seq} \mid T)} > 1 \,\middle|\, H \right) 
+ \frac{1}{2} \cdot \mathbb{P}\!\left( \frac{\mathbb{P}(\text{seq} \mid H)}{\mathbb{P}(\text{seq} \mid T)} = 1 \,\middle|\, H \right).
\end{align*}
$$

Thus, the problem reduces to finding the distribution of  

$$
\frac{\mathbb{P}(\text{seq} \mid H)}{\mathbb{P}(\text{seq} \mid T)}
$$

under the assumption that \(H\) is the true outcome.  

To optimize the solution:
- Observe that a student lying with probability \(p\) can be paired with one lying with probability \(1-p\).
- Use logs.
- Apply the **meet-in-the-middle** approach to reduce complexity.


In [34]:
from collections import defaultdict
from decimal import Decimal, getcontext
from math import log
from bisect import bisect_right

In [44]:
getcontext().prec = 50 

def compute_distribution(probs):
    dp = {Decimal(0): Decimal(1)}
    
    for p in probs:
        p = Decimal(p)
        new_dp = defaultdict(Decimal)
        TT = Decimal(0)
        TF = (Decimal(1) - p) ** 2 / (p ** 2)
        TF = TF.ln() 
        FT = -TF
        FF = Decimal(0)
        
        for val, prob in dp.items():
            new_dp[val + TT] += prob * p * (1 - p)
            new_dp[val + TF] += prob * (1 - p) ** 2
            new_dp[val + FT] += prob * p ** 2
            new_dp[val + FF] += prob * p * (1 - p)
        
        total = sum(new_dp.values())
        for k in new_dp:
            new_dp[k] /= total
        
        dp = new_dp
    return dp

def P(probs):
    n = len(probs)
    half = n // 2
    left_probs = probs[:half]
    right_probs = probs[half:]
    
    left_dp = compute_distribution(left_probs)
    right_dp = compute_distribution(right_probs)
    
    right_keys = sorted(right_dp.keys())
    right_probs_list = [right_dp[k] for k in right_keys]
    prefix = [Decimal(0)]
    
    for p in right_probs_list:
        prefix.append(prefix[-1] + p)
    
    total_prob = Decimal(0)
    total_sum = prefix[-1]
    
    for llr_left, prob_left in left_dp.items():
        threshold = -llr_left
        idx = bisect_right(right_keys, threshold)
        contrib = (total_sum - prefix[idx]) * prob_left
        total_prob += contrib
        
        if idx > 0 and right_keys[idx - 1] == threshold:
            tie_prob = right_dp[right_keys[idx - 1]]
            total_prob += Decimal('0.5') * prob_left * tie_prob
    
    return float(total_prob)  


In [45]:
P([0.2, 0.4])

0.832

In [46]:
probs = [i/100 for i in range(25, 50)]
round(P(probs), 10)

0.9861343531

In [ ]:
0.9861343531